<h3>Introduction</h3>

This project is a practical way to up csv data on MinIO buckets.

On the next cells we go install the dependences of this project (pandas and minio libs) and import all libs hwo we need.

In [ ]:
#!/usr/bin/env python3
! pip install minio==7.1.15
! pip install pandas==2.0.2

In [1]:
import os
import pathlib
from io import BytesIO
from urllib.parse import urlparse

import pandas as pd
from minio import Minio

Here we go define the enviropments variables to be used on MinioSender class. The MinioSender() is the class for manage data using Minio SDK.

In [8]:
MINIO_URL= '' # we dont need use http here, try some like 'localhost:9001'
MINIO_ACCESS_TOKEN= ''
MINIO_SECRET_TOKEN= ''

In [9]:
class MinioSender():

    def __init__(self, url: str, access: str, secret: str) -> None:
        self.url: str = url
        self.access: str = access
        self.secret: str = secret
        self.secure: bool = False

    def minio_client(self) -> Minio:
        client: Minio = Minio(
                self.url,
                access_key=self.access,
                secret_key=self.secret,
                secure=self.secure
            )

        return client

    def get_file_name(self, file_path: str) -> str:
        filename: str = os.path.basename(urlparse(file_path).path)
        return filename

    def send_csv_file(self, file_path: str, bucket_name: str) -> None:
        print(f'sending: {file_path} to {bucket_name} bucket!')
        client: Minio = self.minio_client()
        df = pd.read_csv(
                filepath_or_buffer=str(file_path),
                encoding='iso-8859-1',
                sep=';',
                header=None,
                index_col=False,
                low_memory=False
            )
        csv_bytes = df.to_csv().encode('utf-8')
        csv_buffer = BytesIO(csv_bytes)

        try:
            client.put_object(
                    bucket_name=bucket_name,
                    object_name=str(self.get_file_name(file_path)),
                    data=csv_buffer,
                    length=len(csv_bytes),
                    content_type='application/csv'
                )
        except Exception as exc:
            print(exc)

    def send_all_csv_files(self, folder_path: str, bucket_name: str) -> None:
        folder_addr = pathlib.Path(folder_path)
        for file in list(folder_addr.rglob('*csv')):
            self.send_csv_file(file_path=str(file), bucket_name=bucket_name)


Here we inicalizing the MinioSender object and need define witch bucket we go work.

In [10]:
sender = MinioSender(url=MINIO_URL, access=MINIO_ACCESS_TOKEN, secret=MINIO_SECRET_TOKEN)
bucket_name = '' # write the bucket name from your minIO here

To send all csv files to the bucket run the next cell

In [ ]:
folder_path:str = '' # write here the folder path to .csv files
sender.send_all_csv_files(folder_path=folder_path, bucket_name=bucket_name)

To send an single csv file run the next cell

In [ ]:
file_path:str = '' # write here the file path to .csv files
sender.send_csv_file(file_path=file_path, bucket_name=bucket_name)